In [19]:
import datetime
import stock_marker_binance
import model_design
import pandas as pd
from sklearn.utils import class_weight
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE


In [20]:
EPOCHS =  2000
TIME_FRAME_LIST = [2, 3, 6, 12]
TRAIN_FRACTION = 0.75

In [21]:
def dataframe_to_tensor(df, timestep_length: int):
    
    '''
    Converts a pandas dataframe to a tensor
    
    Input: Pandas dataframe
            timestep_length (int) - the length of the timestep
    Output: Tensor
    
    '''
    
    # Convert the dataframe to a numpy array
    df = np.array(df)

    n_seq = len(df) - timestep_length + 1
    return np.array([df[i:(i+timestep_length)] for i in range(n_seq)])

In [22]:
def smote(df, train_frac):
    '''
    Oversample the minority class using SMOTE prior to OHE.
    '''

    n_train = int(df.shape[0] * train_frac)
    n_test = df.shape[0] - n_train

    train_data = df.iloc[:n_train, :]
    test_data = df.iloc[n_train:, :]

    x_train = train_data.drop(columns = ['action'])
    y_train = train_data['action']

    x_test = test_data.drop(columns = ['action'])
    y_test = test_data['action']


    smote_inst = SMOTE(sampling_strategy='auto')
    
    
    
    x_train , y_train = smote_inst.fit_resample(x_train, y_train)
    
    train_data = pd.concat([x_train, y_train], axis=1)
    test_data = pd.concat([x_test, y_test], axis=1)

    df = pd.concat([train_data, test_data], axis=0)

    return df, n_test

In [23]:
def OneHotEncoding(df_result):
    enc = OneHotEncoder()

    OHE_array = enc.fit_transform(df_result[['action']]).toarray()

    class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                    classes=np.unique(df_result['action']),
                                                    y=df_result['action'])



    class_weights_dict = dict(zip(enc.categories_[0], class_weights))

    print(class_weights)
    print(class_weights_dict.keys())

    
    OHE_df = pd.DataFrame(OHE_array, columns=enc.categories_[0])
    df_result = pd.merge(df_result, OHE_df, left_index=True, right_index=True)
    df_result.drop(columns=['action'], inplace=True)
    return df_result, class_weights_dict

In [24]:
def get_train_test_sets(data, train_frac):
    
    n_train = int(data.shape[0] * train_frac)
    
    x_train = data[:n_train, :, :-4]
    y_train = data[:n_train, -1:, -4:].reshape(-1,4)
    x_test = data[n_train:, :, :-4]
    y_test = data[n_train:, -1:, -4:].reshape(-1,4)
    return x_train, y_train, x_test, y_test

In [25]:
# import tensorflow as tf
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [26]:
df = pd.read_csv('ETH_1h_history as of 09-01-2024.csv', index_col=0)

In [27]:
df.head()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades
0,01/01/2020 00:00,129.16,129.19,128.68,128.87,7769.17336,1.577840e+12,1.000930e+06,2504
1,01/01/2020 01:00,128.87,130.65,128.78,130.64,11344.65516,1.577840e+12,1.474278e+06,4885
2,01/01/2020 02:00,130.63,130.98,130.35,130.85,7603.35623,1.577850e+12,9.940256e+05,3046
3,01/01/2020 03:00,130.85,130.89,129.94,130.20,4968.55433,1.577850e+12,6.473610e+05,2818
4,01/01/2020 04:00,130.21,130.74,130.15,130.20,3397.90747,1.577850e+12,4.430067e+05,2264


In [28]:
df.dtypes

timestamp              object
open                  float64
high                  float64
low                   float64
close                 float64
volume                float64
close_time            float64
quote_asset_volume    float64
num_trades              int64
dtype: object

In [29]:
df_result = stock_marker_binance.sorting_timestamp(df)

df_result['hour_sin'] = np.sin(df_result['timestamp'].dt.hour / 24 * 2 * np.pi)
df_result['hour_cos'] = np.cos(df_result['timestamp'].dt.hour / 24 * 2 * np.pi)

df_result['day_sin'] = np.sin(df_result['timestamp'].dt.day / 30 * 2 * np.pi)
df_result['day_cos'] = np.cos(df_result['timestamp'].dt.day / 30 * 2 * np.pi)

df_result['mon_sin'] = np.sin(df_result['timestamp'].dt.month / 12 * 2 * np.pi)
df_result['mon_cos'] = np.cos(df_result['timestamp'].dt.month / 12 * 2 * np.pi)

df_result['weekday_sin'] = np.sin(df_result['timestamp'].dt.weekday / 7 * 2 * np.pi)
df_result['weekday_cos'] = np.cos(df_result['timestamp'].dt.weekday / 7 * 2 * np.pi)

df_result['year'] = df_result['timestamp'].dt.year


In [30]:
df_result.head(10)

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year
0,2020-01-01 00:00:00,129.16,129.19,128.68,128.87,7769.17336,1.577840e+12,1.000930e+06,2504,0.000000,1.000000e+00,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
1,2020-01-01 01:00:00,128.87,130.65,128.78,130.64,11344.65516,1.577840e+12,1.474278e+06,4885,0.258819,9.659258e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
2,2020-01-01 02:00:00,130.63,130.98,130.35,130.85,7603.35623,1.577850e+12,9.940256e+05,3046,0.500000,8.660254e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
3,2020-01-01 03:00:00,130.85,130.89,129.94,130.20,4968.55433,1.577850e+12,6.473610e+05,2818,0.707107,7.071068e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
4,2020-01-01 04:00:00,130.21,130.74,130.15,130.20,3397.90747,1.577850e+12,4.430067e+05,2264,0.866025,5.000000e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
5,2020-01-01 05:00:00,130.20,130.47,130.11,130.30,4243.60640,1.577860e+12,5.529727e+05,2426,0.965926,2.588190e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
6,2020-01-01 06:00:00,130.31,130.75,130.26,130.44,3668.90166,1.577860e+12,4.789442e+05,2347,1.000000,6.123234e-17,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
7,2020-01-01 07:00:00,130.47,130.71,130.14,130.24,4147.17413,1.577870e+12,5.407707e+05,2568,0.965926,-2.588190e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
8,2020-01-01 08:00:00,130.24,130.41,129.87,130.36,7541.44497,1.577870e+12,9.808940e+05,3039,0.866025,-5.000000e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
9,2020-01-01 09:00:00,130.40,130.62,130.13,130.17,4808.20496,1.577870e+12,6.268190e+05,2696,0.707107,-7.071068e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020


In [31]:

df_result = stock_marker_binance.std(df_result, ['close', 'quote_asset_volume', 'num_trades'])
print("Complete adding STDs")

df_result = stock_marker_binance.adding_MAs(df_result, ['close', 'quote_asset_volume', 'num_trades'])
print("Complete adding MAs")


df_result = stock_marker_binance.adding_ratio(df_result, ['close', 'quote_asset_volume', 'num_trades'], ['MA'])
df_result = stock_marker_binance.adding_ratio(df_result, ['close'], ['STD'])
print("Complete adding ratios")

df_result = stock_marker_binance.rsi(df_result, ['close', 
                                                #  'quote_asset_volume', 'num_trades'
                                                 ])
print("Complete adding RSI")

df_result = stock_marker_binance.marking_buy_sell_actions(df_result)
print("Complete marking buy sell actions")

df_result = stock_marker_binance.marking_adjacent_actions(df_result)
print("Complete marking adjacent actions")

df_result = stock_marker_binance.marking_hold_wait_actions(df_result)
print("Complete marking hold wait actions")

df_result = df_result.drop(columns=[
                                    'timestamp',
                                    'open', 'high', 'low' ,'close_time', 'volume',
                                    'close',  
                                    'quote_asset_volume', 'num_trades'
                                    ])
df_result=df_result.dropna()
df_result.reset_index(inplace=True, drop=True)
# df_result = stock_marker_binance.renaming_cols(df_result, STOCK_TICKER + "_", ['timestamp', 'action'])

Complete adding STDs
Complete adding MAs
Complete adding ratios
Complete adding RSI


Complete marking buy sell actions
Complete marking adjacent actions
Complete marking hold wait actions


In [32]:
df_result.groupby('action').count()

,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,STD3_close,...,RATIO_close_and_STD192,RATIO_close_and_STD384,RSI_3_close,RSI_6_close,RSI_12_close,RSI_24_close,RSI_48_close,RSI_96_close,RSI_192_close,RSI_384_close
action,,,,,,,,,,,,,,,,,,,,,
B,2332,2332,2332,2332,2332,2332,2332,2332,2332,2332,...,2332,2332,2332,2332,2332,2332,2332,2332,2332,2332
H,11110,11110,11110,11110,11110,11110,11110,11110,11110,11110,...,11110,11110,11110,11110,11110,11110,11110,11110,11110,11110
S,2628,2628,2628,2628,2628,2628,2628,2628,2628,2628,...,2628,2628,2628,2628,2628,2628,2628,2628,2628,2628
W,18778,18778,18778,18778,18778,18778,18778,18778,18778,18778,...,18778,18778,18778,18778,18778,18778,18778,18778,18778,18778


In [33]:
# Dropping cols
drop_cols_list =[]
for col in df_result.columns:
    if col[:2]=="MA" or col[:3]=="STD":
        drop_cols_list.append(col)
df_result.drop(columns=drop_cols_list, inplace=True)

In [34]:
df_result

,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,RATIO_close_and_MA3,...,RATIO_close_and_STD384,RSI_3_close,RSI_6_close,RSI_12_close,RSI_24_close,RSI_48_close,RSI_96_close,RSI_192_close,RSI_384_close,action
0,0.000000,1.000000e+00,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,0.998395,...,15.673192,65.600000,53.818828,56.451613,49.606299,45.735591,58.575993,56.921027,56.004265,B
1,0.258819,9.659258e-01,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.003510,...,15.709274,72.784810,59.938367,66.561514,61.406250,48.108449,58.861391,56.921027,55.899593,S
2,0.500000,8.660254e-01,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.005462,...,15.713150,81.465517,51.035782,67.384615,63.572149,48.991400,60.461694,57.310652,55.990641,W
3,0.707107,7.071068e-01,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,0.999496,...,15.599209,78.099174,73.841962,63.047809,63.284782,52.077793,60.000000,57.287449,56.013769,W
4,0.866025,5.000000e-01,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.005046,...,15.667762,81.468531,78.378378,71.428571,65.007180,55.126699,60.382340,57.649960,56.242723,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34843,0.707107,7.071068e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,0.999538,...,42.424458,63.647491,14.575412,62.740044,72.020706,57.083589,54.378381,51.021152,49.886262,W
34844,0.866025,5.000000e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,1.001322,...,42.530161,74.787776,25.061152,66.604418,72.069594,57.045402,54.645727,51.030496,50.098711,W
34845,0.965926,2.588190e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,0.999021,...,42.404276,39.237233,25.491970,50.822577,70.374137,56.957313,54.037629,50.948907,50.349035,W
34846,1.000000,6.123234e-17,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,0.997117,...,42.276558,31.499741,41.071429,32.320816,66.723177,55.541223,53.031366,50.579826,50.167989,W


In [35]:
# Do NOT use SMOTE for LSTM!!!
# df_result, n_test = smote(df_result, TRAIN_FRACTION)

In [17]:
df_result.head()

,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,RATIO_close_and_MA3,...,RATIO_close_and_STD384,RSI_3_close,RSI_6_close,RSI_12_close,RSI_24_close,RSI_48_close,RSI_96_close,RSI_192_close,RSI_384_close,action
0,0.000000,1.000000,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,0.998395,...,15.673192,65.600000,53.818828,56.451613,49.606299,45.735591,58.575993,56.921027,56.004265,B
1,0.258819,0.965926,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.003510,...,15.709274,72.784810,59.938367,66.561514,61.406250,48.108449,58.861391,56.921027,55.899593,S
2,0.500000,0.866025,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.005462,...,15.713150,81.465517,51.035782,67.384615,63.572149,48.991400,60.461694,57.310652,55.990641,W
3,0.707107,0.707107,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,0.999496,...,15.599209,78.099174,73.841962,63.047809,63.284782,52.077793,60.000000,57.287449,56.013769,W
4,0.866025,0.500000,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.005046,...,15.667762,81.468531,78.378378,71.428571,65.007180,55.126699,60.382340,57.649960,56.242723,W


In [18]:
df_result, class_weights_dict = OneHotEncoding(df_result)

[3.73584906 0.78415842 3.31506849 0.46394717]
dict_keys(['B', 'H', 'S', 'W'])


In [19]:
df_result.head()

,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,RATIO_close_and_MA3,...,RSI_12_close,RSI_24_close,RSI_48_close,RSI_96_close,RSI_192_close,RSI_384_close,B,H,S,W
0,0.000000,1.000000,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,0.998395,...,56.451613,49.606299,45.735591,58.575993,56.921027,56.004265,1.0,0.0,0.0,0.0
1,0.258819,0.965926,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.003510,...,66.561514,61.406250,48.108449,58.861391,56.921027,55.899593,0.0,0.0,1.0,0.0
2,0.500000,0.866025,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.005462,...,67.384615,63.572149,48.991400,60.461694,57.310652,55.990641,0.0,0.0,0.0,1.0
3,0.707107,0.707107,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,0.999496,...,63.047809,63.284782,52.077793,60.000000,57.287449,56.013769,0.0,0.0,0.0,1.0
4,0.866025,0.500000,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.005046,...,71.428571,65.007180,55.126699,60.382340,57.649960,56.242723,0.0,0.0,0.0,1.0


In [20]:
df_result.columns

Index(['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'mon_sin', 'mon_cos',
       'weekday_sin', 'weekday_cos', 'year', 'RATIO_close_and_MA3',
       'RATIO_close_and_MA6', 'RATIO_close_and_MA12', 'RATIO_close_and_MA24',
       'RATIO_close_and_MA48', 'RATIO_close_and_MA96', 'RATIO_close_and_MA192',
       'RATIO_close_and_MA384', 'RATIO_quote_asset_volume_and_MA3',
       'RATIO_quote_asset_volume_and_MA6', 'RATIO_quote_asset_volume_and_MA12',
       'RATIO_quote_asset_volume_and_MA24',
       'RATIO_quote_asset_volume_and_MA48',
       'RATIO_quote_asset_volume_and_MA96',
       'RATIO_quote_asset_volume_and_MA192',
       'RATIO_quote_asset_volume_and_MA384', 'RATIO_num_trades_and_MA3',
       'RATIO_num_trades_and_MA6', 'RATIO_num_trades_and_MA12',
       'RATIO_num_trades_and_MA24', 'RATIO_num_trades_and_MA48',
       'RATIO_num_trades_and_MA96', 'RATIO_num_trades_and_MA192',
       'RATIO_num_trades_and_MA384', 'RATIO_close_and_STD3',
       'RATIO_close_and_STD6', 'RATIO_close_a

In [21]:
max_val_PR = 0.65

for time_step in TIME_FRAME_LIST:
    df_tensor = dataframe_to_tensor(df_result, time_step)
    x_train, y_train, x_test, y_test = get_train_test_sets(df_tensor, TRAIN_FRACTION)
    
    df_tensor.shape

    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)
    model, history = model_design.model_design(EPOCHS,  x_train, y_train, x_test, y_test, class_weights_dict)

    val_PR = round(max(history.history['val_sens_at_spec']),3)
    
    x = datetime.datetime.now()
    
    model_name = "models/" + x.strftime("%Y-%m-%d %H-%M-%S") +" val_sens_at_spec " + str(val_PR) + " time frame " + str(time_step) + ".hd5"

    if val_PR > max_val_PR:
        max_val_PR = val_PR
        model.save(model_name,  save_format = 'h5')
    
    print(model_name)
    print(val_PR)
    print(150 * "-")


(26135, 2, 49)
(26135, 4)
(8712, 2, 49)
(8712, 4)





models/2024-02-09 12-30-19 val_sens_at_spec 0.576 time frame 2.hd5
0.576
------------------------------------------------------------------------------------------------------------------------------------------------------
(26134, 3, 49)
(26134, 4)
(8712, 3, 49)
(8712, 4)
models/2024-02-09 12-39-01 val_sens_at_spec 0.416 time frame 3.hd5
0.416
------------------------------------------------------------------------------------------------------------------------------------------------------
(26132, 6, 49)
(26132, 4)
(8711, 6, 49)
(8711, 4)
models/2024-02-09 12-50-44 val_sens_at_spec 0.513 time frame 6.hd5
0.513
------------------------------------------------------------------------------------------------------------------------------------------------------
(26127, 12, 49)
(26127, 4)
(8710, 12, 49)
(8710, 4)
models/2024-02-09 13-07-45 val_sens_at_spec 0.102 time frame 12.hd5
0.102
----------------------------------------------------------------------------------------------------

In [22]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 21, 49)]          0         
                                                                 
 normalization_4 (Normaliza  (None, 21, 49)            99        
 tion)                                                           
                                                                 
 lstm_8 (LSTM)               (None, 21, 147)           115836    
                                                                 
 lstm_9 (LSTM)               (None, 98)                96432     
                                                                 
 dense_8 (Dense)             (None, 98)                9702      
                                                                 
 dense_9 (Dense)             (None, 4)                 396       
                                                           

In [23]:
history.history['val_sens_at_spec']

[0.061674512922763824,
 0.061674512922763824,
 0.061789363622665405,
 0.061674512922763824,
 0.06707247346639633,
 0.06718732416629791,
 0.06684277206659317,
 0.06684277206659317,
 0.06718732416629791,
 0.06776157021522522,
 0.06776157021522522,
 0.0678764209151268,
 0.06776157021522522,
 0.0678764209151268,
 0.06741701811552048,
 0.07419317960739136,
 0.07407832890748978,
 0.0745377317070961,
 0.07419317960739136,
 0.07442288100719452,
 0.07442288100719452,
 0.07442288100719452,
 0.07476742565631866,
 0.07545652985572815,
 0.07557138055562973,
 0.07568623125553131,
 0.07580108195543289,
 0.08188813924789429,
 0.0824623852968216,
 0.08292178809642792,
 0.08292178809642792,
 0.08395543694496155,
 0.08475939184427261,
 0.08498908579349518,
 0.08556333929300308,
 0.08556333929300308,
 0.08625244349241257,
 0.08694154024124146,
 0.08774549514055252,
 0.08843459188938141,
 0.08900884538888931,
 0.08958309143781662,
 0.09073159843683243,
 0.09165039658546448,
 0.10049385577440262,
 0.1012978